In [42]:
import pandas as pd

In [43]:
kaggle_data = pd.read_csv('fromkaggle.csv', )

In [44]:
kaggle_data.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id'],
      dtype='object')

1. Add lables on whether the tweets come from a company or user

In [45]:
whether_tweet_from_com = []

In [46]:
for i in range(0 ,len(kaggle_data)):
    authorID = kaggle_data.at[i, 'author_id']
    authorID = ''.join(j for j in authorID if not j.isdigit())
    if authorID == '':
        whether_tweet_from_com.append(0)
    else:
        whether_tweet_from_com.append(1)

In [47]:
kaggle_data['tweet_from_company'] = whether_tweet_from_com

2. Based on the tweet's author, further generate the label

   1) The response should be from company
   
   2) The questions should be from users

In [48]:
kaggle_data_fromcom = kaggle_data[kaggle_data.tweet_from_company == 1]

In [49]:
response_text = kaggle_data_fromcom[kaggle_data_fromcom.in_response_to_tweet_id.notnull()].text
response_tweet_inres = kaggle_data_fromcom[kaggle_data_fromcom.in_response_to_tweet_id.notnull()].in_response_to_tweet_id
tweet_ids = kaggle_data_fromcom[kaggle_data_fromcom.in_response_to_tweet_id.notnull()].tweet_id

In [50]:
print(len(response_text))
print(len(response_tweet_inres))

1266942
1266942


In [51]:
response_text_small = response_text.apply(lambda x: str(x).lower())

In [52]:
non_response_list = [['dm', 'me'], ['dm', 'us'], ['please', 'share'], ['we', 'dm'], ['send us', ''], ['send me', ''], 
                     ['direct message', 'me'],['direct message', 'us'],['we','direct message'], ['we', 'send'],
                    ['reach out', 'us']]

In [53]:
response_list = ['account information', 'account number', 'full name', 'email address', 'reservation']

In [54]:
def check_non_response(non_response_list, text):
    for vec in non_response_list:
        if vec[0] in text and vec[1] in text:
            return True
    return False

In [55]:
def check_response(response_list, text):
    for string in response_list:
        if string in text:
            return True
    return False

In [56]:
response_ability_labels = {}

In [57]:
for row_num in range(0, len(response_text_small)):
    text = str(response_text_small.iloc[row_num])
    inres = str(int(response_tweet_inres.iloc[row_num]))
    tweetid = str(int(tweet_ids.iloc[row_num]))
    
    if check_non_response(non_response_list, text) == True and check_response(response_list, text) == False:
        response_ability_labels[inres] = 0
    else:
        response_ability_labels[inres] = 1

3. Mark the labelling back to kaggle dataframe with question author id not company

In [58]:
response_ability_labels_list = []

In [59]:
for row_num in range(0, len(kaggle_data)):
    from_com = kaggle_data.tweet_from_company.iloc[row_num]
    tweetid = str(kaggle_data.tweet_id.iloc[row_num])
    if tweetid in response_ability_labels and from_com == 0:
        response_ability_labels_list.append(response_ability_labels[tweetid])
    else:
        response_ability_labels_list.append('nan')

In [60]:
kaggle_data['response_ability_label'] = response_ability_labels_list

In [61]:
kaggle_data.columns

Index(['tweet_id', 'author_id', 'inbound', 'created_at', 'text',
       'response_tweet_id', 'in_response_to_tweet_id', 'tweet_from_company',
       'response_ability_label'],
      dtype='object')

In [62]:
kaggle_data.to_csv('./kaggle_inter.csv', index=False) # without filtering out any data ---- Used in second clustering

4. Filter out the 'NaN' data

In [21]:
kaggle_data_labeled = kaggle_data[kaggle_data.response_ability_label != 'nan']

In [22]:
len(kaggle_data_labeled)

1164365

5. Rebalance the data from different classes

In [99]:
Train_able = kaggle_data_labeled.loc[kaggle_data_labeled.response_ability_label == 1]

In [100]:
Train_disable = kaggle_data_labeled.loc[kaggle_data_labeled.response_ability_label == 0]

In [103]:
print(len(Train_disable))
print(len(Train_able))

295252
869113


In [104]:
Train_able = Train_able.sample(n=295252)

In [105]:
train_data = Train_able.append(Train_disable)

6. Output final csv after labelling

In [106]:
train_data.to_csv('./kaggle_labeled.csv', index = False)